# Project: Fraud Detection 

## 1. Overview

### PaySim simulates mobile money transactions based on a sample of real transacions extracted from one month of financial logs from a mobile money service implemented in an African country. The original logs were provided by a multinational company, who is the provider of the mobile financial service which is currently running in more than 14 countries all around the world. The objective of the project is to predict if a transaction is fraudulent or not.

## 2. Preprocess the data

#### We´ll use PySpark to preprocess the data.

In [226]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.functions import vector_to_array
import functools

In [173]:
# global variables

global df_bank, results 

In [174]:
# creation of the SparkSession

spark = SparkSession.builder.appName("FraudDetection").getOrCreate()
spark

In [175]:
# spark dataframe 

df = spark.read.csv('fraudDetection.csv', header=True)

#### Let´s take a look to the data with the first 10 rows.

In [176]:
df.show(10)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|      

In [177]:
df.printSchema()

root
 |-- step: string (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: string (nullable = true)
 |-- newbalanceOrig: string (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: string (nullable = true)
 |-- newbalanceDest: string (nullable = true)
 |-- isFraud: string (nullable = true)
 |-- isFlaggedFraud: string (nullable = true)



#### There are 11 columns, some of them are numerical and others are categorical. Let´s count the number of registers.

In [178]:
print(f"The total number of registers is:",df.count())

The total number of registers is: 6362620


#### We have more than six miliions of transactions in the dataset.

### 2.1 Feature Engineering

#### Firstly, we´ll create a function to create a new variable.

In [179]:
### 1.- creation of a new variable: type2

df.printSchema()

root
 |-- step: string (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: string (nullable = true)
 |-- newbalanceOrig: string (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: string (nullable = true)
 |-- newbalanceDest: string (nullable = true)
 |-- isFraud: string (nullable = true)
 |-- isFlaggedFraud: string (nullable = true)



In [180]:
type(df)

pyspark.sql.dataframe.DataFrame

In [181]:
df.show()

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT|  9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT|  1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|    181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|    181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT| 11668.14|C2048537720|      41554.0|      29885.86|M123070170

In [182]:
### creation of a view of the table named "fraud"

df.createOrReplaceTempView("fraud")

In [183]:
df.count()

6362620

In [184]:
#  use sparkSQL to query and form the column "type2", which concatenates the first letter of each of those 
#  columns (nameOrig and nameDest)

type2 = spark.sql("SELECT SUBSTRING(nameOrig,1,1) || '' || SUBSTRING(nameDest,1,1) as type2 FROM fraud ")
type2.show(10)

+-----+
|type2|
+-----+
|   CM|
|   CM|
|   CC|
|   CC|
|   CM|
|   CM|
|   CM|
|   CM|
|   CM|
|   CC|
+-----+
only showing top 10 rows



In [185]:
#  convert the previous spark dataframe into a pandas one

col_type2 = type2.toPandas()

In [186]:
type(col_type2)

pandas.core.frame.DataFrame

In [187]:
### 2.1.- One Hot Encoding: column "type"

df.show(3)

+----+--------+-------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type| amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+-------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT|9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT|1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|  181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
+----+--------+-------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
only showing top 3 rows



#### We´ll use some libraries of Spark for Machine Learning (SparkML).

In [188]:
### StringIndexer Initialization
### column: type

indexer_type = StringIndexer(inputCol="type",outputCol="types_indexed")
indexerModel_type = indexer_type.fit(df)


In [189]:
### Transform the DataFrame using the fitted StringIndexer model

indexed_df_type = indexerModel_type.transform(df)
indexed_df_type.show(10)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+-------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|types_indexed|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+-------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|          1.0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|          1.0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|          3.0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|

#### Here, we´ve set each of the elements of the "type" column into indexes.

In [190]:
### apply One-Hot-Encoding to the indexed column, that is, 
### "types_indexed"

encoder_type = OneHotEncoder(dropLast=False, inputCol="types_indexed", outputCol="types_onehot")
encoder_type_df = encoder_type.fit(indexed_df_type).transform(indexed_df_type)
encoder_type_df.show(truncate=False)


+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+-------------+-------------+
|step|type    |amount   |nameOrig   |oldbalanceOrg|newbalanceOrig|nameDest   |oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|types_indexed|types_onehot |
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+-------------+-------------+
|1   |PAYMENT |9839.64  |C1231006815|170136.0     |160296.36     |M1979787155|0.0           |0.0           |0      |0             |1.0          |(5,[1],[1.0])|
|1   |PAYMENT |1864.28  |C1666544295|21249.0      |19384.72      |M2044282225|0.0           |0.0           |0      |0             |1.0          |(5,[1],[1.0])|
|1   |TRANSFER|181.0    |C1305486145|181.0        |0.0           |C553264065 |0.0           |0.0           |1      |0             |3.0          |(5,[3],[1.0])|
|1   |CASH_OUT|181.0    |C840083671 |181

In [191]:
encoder_type_df.printSchema()

root
 |-- step: string (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: string (nullable = true)
 |-- newbalanceOrig: string (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: string (nullable = true)
 |-- newbalanceDest: string (nullable = true)
 |-- isFraud: string (nullable = true)
 |-- isFlaggedFraud: string (nullable = true)
 |-- types_indexed: double (nullable = false)
 |-- types_onehot: vector (nullable = true)



In [192]:
encoder_type_df_split = encoder_type_df.select('*',vector_to_array('types_onehot').alias('types_onehot_split'))
encoder_type_df_split.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+-------------+-------------+--------------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|types_indexed| types_onehot|  types_onehot_split|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+-------------+-------------+--------------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|          1.0|(5,[1],[1.0])|[0.0, 1.0, 0.0, 0...|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|          1.0|(5,[1],[1.0])|[0.0, 1.0, 0.0, 0...|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|        

In [193]:
### now, we´ll split the "types_onehot_split" into 

num_categories = len(encoder_type_df_split.first()['types_onehot_split'])
cols_expanded = [(f.col('types_onehot_split')[i].alias(f"{indexerModel_type.labels[i]}")) for i in range(num_categories)]
type_df = encoder_type_df_split.select('*',*cols_expanded)


In [194]:
type_df.show(100)

+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+-------------+-------------+--------------------+--------+-------+-------+--------+-----+
|step|    type|    amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|types_indexed| types_onehot|  types_onehot_split|CASH_OUT|PAYMENT|CASH_IN|TRANSFER|DEBIT|
+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+-------------+-------------+--------------------+--------+-------+-------+--------+-----+
|   1| PAYMENT|   9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|          1.0|(5,[1],[1.0])|[0.0, 1.0, 0.0, 0...|     0.0|    1.0|    0.0|     0.0|  0.0|
|   1| PAYMENT|   1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|   

In [195]:
### 2.2.- One Hot Encoding: column "type2"

type2.show(5)

+-----+
|type2|
+-----+
|   CM|
|   CM|
|   CC|
|   CC|
|   CM|
+-----+
only showing top 5 rows



In [196]:
### StringIndexer Initialization
### column: type2

indexer_type = StringIndexer(inputCol="type2",outputCol="types_indexed")
indexerModel_type = indexer_type.fit(type2)

In [197]:
### Transform the DataFrame using the fitted StringIndexer model

indexed_df_type = indexerModel_type.transform(type2)
indexed_df_type.show(10)

+-----+-------------+
|type2|types_indexed|
+-----+-------------+
|   CM|          1.0|
|   CM|          1.0|
|   CC|          0.0|
|   CC|          0.0|
|   CM|          1.0|
|   CM|          1.0|
|   CM|          1.0|
|   CM|          1.0|
|   CM|          1.0|
|   CC|          0.0|
+-----+-------------+
only showing top 10 rows



In [198]:
### apply One-Hot-Encoding to the indexed column, that is, 
### "types_indexed"

encoder_type2 = OneHotEncoder(dropLast=False, inputCol="types_indexed", outputCol="types_onehot")
encoder_type2_df = encoder_type.fit(indexed_df_type).transform(indexed_df_type)
encoder_type2_df.show(truncate=False)

+-----+-------------+-------------+
|type2|types_indexed|types_onehot |
+-----+-------------+-------------+
|CM   |1.0          |(2,[1],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CC   |0.0          |(2,[0],[1.0])|
|CC   |0.0          |(2,[0],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CC   |0.0          |(2,[0],[1.0])|
|CC   |0.0          |(2,[0],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CC   |0.0          |(2,[0],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CM   |1.0          |(2,[1],[1.0])|
|CC   |0.0          |(2,[0],[1.0])|
+-----+-------------+-------------+
only showing top 20 rows



In [199]:
encoder_type2_df.printSchema()

root
 |-- type2: string (nullable = true)
 |-- types_indexed: double (nullable = false)
 |-- types_onehot: vector (nullable = true)



In [200]:
encoder_type2_df_split = encoder_type2_df.select('*',vector_to_array('types_onehot').alias('types_onehot_split'))
encoder_type2_df_split.show(5)

+-----+-------------+-------------+------------------+
|type2|types_indexed| types_onehot|types_onehot_split|
+-----+-------------+-------------+------------------+
|   CM|          1.0|(2,[1],[1.0])|        [0.0, 1.0]|
|   CM|          1.0|(2,[1],[1.0])|        [0.0, 1.0]|
|   CC|          0.0|(2,[0],[1.0])|        [1.0, 0.0]|
|   CC|          0.0|(2,[0],[1.0])|        [1.0, 0.0]|
|   CM|          1.0|(2,[1],[1.0])|        [0.0, 1.0]|
+-----+-------------+-------------+------------------+
only showing top 5 rows



In [201]:
### now, we´ll split the "types_onehot_split" into 

num_categories = len(encoder_type2_df_split.first()['types_onehot_split'])
cols_expanded = [(f.col('types_onehot_split')[i].alias(f"{indexerModel_type.labels[i]}")) for i in range(num_categories)]
encoder_type2_df_split = encoder_type2_df_split.select('*',*cols_expanded)

In [202]:
encoder_type2_df_split.show(5)

+-----+-------------+-------------+------------------+---+---+
|type2|types_indexed| types_onehot|types_onehot_split| CC| CM|
+-----+-------------+-------------+------------------+---+---+
|   CM|          1.0|(2,[1],[1.0])|        [0.0, 1.0]|0.0|1.0|
|   CM|          1.0|(2,[1],[1.0])|        [0.0, 1.0]|0.0|1.0|
|   CC|          0.0|(2,[0],[1.0])|        [1.0, 0.0]|1.0|0.0|
|   CC|          0.0|(2,[0],[1.0])|        [1.0, 0.0]|1.0|0.0|
|   CM|          1.0|(2,[1],[1.0])|        [0.0, 1.0]|0.0|1.0|
+-----+-------------+-------------+------------------+---+---+
only showing top 5 rows



In [203]:
type(encoder_type2_df_split)

pyspark.sql.dataframe.DataFrame

In [204]:
type2 = encoder_type2_df_split.drop("type2","types_onehot","types_indexed","types_onehot_split")

In [205]:
type2.show(5)

+---+---+
| CC| CM|
+---+---+
|0.0|1.0|
|0.0|1.0|
|1.0|0.0|
|1.0|0.0|
|0.0|1.0|
+---+---+
only showing top 5 rows



In [209]:
df.columns

['step',
 'type',
 'amount',
 'nameOrig',
 'oldbalanceOrg',
 'newbalanceOrig',
 'nameDest',
 'oldbalanceDest',
 'newbalanceDest',
 'isFraud',
 'isFlaggedFraud']

In [ ]:
### 2.3.- Eliminate unneccesary columns: "nameOrig","nameDest","isFlaggedFraud","newbalanceDest",
### "oldbalanceDest","oldbalanceOrg","newbalanceOrig"


#### Now, we´ll eliminate the unnecessary columns:
+ nameOrig
+ nameDest
+ isFlaggedFraud
+ newbalanceDest
+ oldbalanceDest
+ oldbalanceOrg
+ newbalanceOrig 

In [212]:
type_df = type_df.drop("nameOrig","nameDest","isFlaggedFraud","newbalanceDest","oldbalanceDest","oldbalanceOrg","newbalanceOrig","type","types_indexed","types_onehot","types_onehot_split")
type_df.show(5)

+----+--------+-------+--------+-------+-------+--------+-----+
|step|  amount|isFraud|CASH_OUT|PAYMENT|CASH_IN|TRANSFER|DEBIT|
+----+--------+-------+--------+-------+-------+--------+-----+
|   1| 9839.64|      0|     0.0|    1.0|    0.0|     0.0|  0.0|
|   1| 1864.28|      0|     0.0|    1.0|    0.0|     0.0|  0.0|
|   1|   181.0|      1|     0.0|    0.0|    0.0|     1.0|  0.0|
|   1|   181.0|      1|     1.0|    0.0|    0.0|     0.0|  0.0|
|   1|11668.14|      0|     0.0|    1.0|    0.0|     0.0|  0.0|
+----+--------+-------+--------+-------+-------+--------+-----+
only showing top 5 rows



In [249]:
type2.show(5)

+---+---+
| CC| CM|
+---+---+
|0.0|1.0|
|0.0|1.0|
|1.0|0.0|
|1.0|0.0|
|0.0|1.0|
+---+---+
only showing top 5 rows



In [250]:
type(type2)

pyspark.sql.dataframe.DataFrame

In [248]:
type_df.join

<bound method DataFrame.join of DataFrame[step: string, amount: string, isFraud: string, CASH_OUT: double, PAYMENT: double, CASH_IN: double, TRANSFER: double, DEBIT: double]>

In [206]:
import pandas as pd

In [207]:
pd.read_csv("fraudDetection.csv")

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [208]:
# function: process_data()

def process_data(df):
    global df_bank
    df.

    

SyntaxError: invalid syntax (3553925290.py, line 5)

#### We´ll eliminate some columns that are irrelevant in this process:
+ nameOrig
+ nameDest
+ isFlaggedFraud
+ oldbalanceOrg
+ newbalanceOrg
+ oldbalanceDest
+ newbalanceDest


In [ ]:
df.printSchema()

root
 |-- step: string (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: string (nullable = true)
 |-- newbalanceOrig: string (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: string (nullable = true)
 |-- newbalanceDest: string (nullable = true)
 |-- isFraud: string (nullable = true)
 |-- isFlaggedFraud: string (nullable = true)



In [ ]:
# elimination of unneccesary columns

df = df.drop('nameDest','nameOrig','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','isFlaggedFraud')
df.show(10)

+----+--------+--------+-------+
|step|    type|  amount|isFraud|
+----+--------+--------+-------+
|   1| PAYMENT| 9839.64|      0|
|   1| PAYMENT| 1864.28|      0|
|   1|TRANSFER|   181.0|      1|
|   1|CASH_OUT|   181.0|      1|
|   1| PAYMENT|11668.14|      0|
|   1| PAYMENT| 7817.71|      0|
|   1| PAYMENT| 7107.77|      0|
|   1| PAYMENT| 7861.64|      0|
|   1| PAYMENT| 4024.36|      0|
|   1|   DEBIT| 5337.77|      0|
+----+--------+--------+-------+
only showing top 10 rows



In [ ]:
df.printSchema()

root
 |-- step: string (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: string (nullable = true)
 |-- newbalanceOrig: string (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: string (nullable = true)
 |-- newbalanceDest: string (nullable = true)
 |-- isFraud: string (nullable = true)
 |-- isFlaggedFraud: string (nullable = true)

